In [46]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## Analyze the Data

可以看到cabin的na很多，ticket的意义不大，这两个feature直接丢掉。

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
data.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


name 里面有个title值得提取，可能会有有用的信息。

In [22]:
data['Titles'] = [re.search(r'\b([A-Za-z]+)\.', name)[0] for name in data['Name']]

In [ ]:
data.drop(['Name'], axis=1, inplace=True)

In [28]:
data.describe(include='all')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titles
count,891.000000,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,889,891
unique,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,3,17
top,NaN,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Mr.
freq,NaN,NaN,NaN,577,NaN,NaN,NaN,NaN,644,517
mean,446.000000,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,14.526497,1.102743,0.806057,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,20.125000,0.000000,0.000000,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,28.000000,0.000000,0.000000,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,38.000000,1.000000,0.000000,31.000000,NaN,NaN


我们的特征已经处理好了，里面还有些缺失值，先不管
编码一下字符特征

In [29]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for str_feature in ['Titles', 'Sex', 'Embarked']:
    data[str_feature] = le.fit_transform(data[str_feature])

In [31]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titles
0,1,0,3,1,22.0,1,0,7.2500,2,12
1,2,1,1,0,38.0,1,0,71.2833,0,13
2,3,1,3,0,26.0,0,0,7.9250,2,9
3,4,1,1,0,35.0,1,0,53.1000,2,13
4,5,0,3,1,35.0,0,0,8.0500,2,12


# Train test split

In [34]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.2, shuffle=True, random_state=11)

x = data.drop(['Survived'], axis=1)
y = data['Survived']

x_train, x_test = x.loc[data_train.index], x.loc[data_test.index]
y_train, y_test = y.loc[data_train.index], y.loc[data_test.index]

# Model Training

## LGBM

In [36]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier()
lgbm_clf.fit(x_train, y_train)

LGBMClassifier()

In [38]:
from sklearn.metrics import accuracy_score

y_pred = lgbm_clf.predict(x_test)
accuracy_score(y_train, lgbm_clf.predict(x_train)), accuracy_score(y_test, y_pred)

(1.0, 0.8212290502793296)

##  XGBOOST

In [48]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [49]:
accuracy_score(y_train, xgb_clf.predict(x_train)), accuracy_score(y_test, xgb_clf.predict(x_test))

(1.0, 0.7988826815642458)

# AUTOGLUON

In [55]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = 'Survived'
predictor = TabularPredictor(label=label).fit(data_train)

No path specified. Models will be saved in: "AutogluonModels\ag-20231202_154218\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231202_154218\"
AutoGluon Version:  0.8.2
Python Version:     3.9.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   9.27 GB / 161.06 GB (5.8%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Train Data Rows:    712
Train Data Columns: 9
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...


In [56]:
predictor.leaderboard(data_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.871508,0.825175,0.010741,0.003905,0.506260,0.010741,0.003905,0.506260,1,True,11
1,LightGBM,0.865922,0.853147,0.002929,0.001951,0.777939,0.002929,0.001951,0.777939,1,True,4
2,CatBoost,0.849162,0.881119,0.003906,0.003905,0.877625,0.003906,0.003905,0.877625,1,True,7
3,WeightedEnsemble_L2,0.849162,0.888112,0.008786,0.006835,1.939866,0.001951,0.000979,0.284302,2,True,14
4,LightGBMXT,0.832402,0.860140,0.001952,0.001953,0.577866,0.001952,0.001953,0.577866,1,True,3
5,NeuralNetTorch,0.832402,0.818182,0.011720,0.010992,4.752714,0.011720,0.010992,4.752714,1,True,12
6,NeuralNetFastAI,0.832402,0.832168,0.011873,0.011716,2.717088,0.011873,0.011716,2.717088,1,True,10
7,ExtraTreesEntr,0.826816,0.818182,0.063056,0.048826,0.505587,0.063056,0.048826,0.505587,1,True,9
8,RandomForestGini,0.826816,0.839161,0.063473,0.045894,0.488134,0.063473,0.045894,0.488134,1,True,5
9,RandomForestEntr,0.826816,0.860140,0.063669,0.055004,0.494301,0.063669,0.055004,0.494301,1,True,6


# Ensemble

In [50]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('lgbm', lgbm_clf), ('xgb', xgb_clf)], voting='soft'
)
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lgbm', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=No

In [51]:
accuracy_score(y_train, voting_clf.predict(x_train)), accuracy_score(y_test, voting_clf.predict(x_test))

(1.0, 0.8212290502793296)